## Sequence Mastery Checkpoint - Joao Cavalcanti

This algorithm generates a t-dimensional matrix for aligning t sequences. Utilizes a very basic scoring, where a pairwise match is 1 point, and everything else 0. Only supports 2-5 sequences. Optimize running or space complexity wasn't a goal of this exercise.

The greatest difficulty was iterating over the matrix and backtracing generically, but it was mainly achievable by using ndarrays and a built-in method that allows iteration with index tracking.

### Helper Functions

In [1]:
%load_ext autoreload
%autoreload 2

import Assignment2_helper as helper1
import Topic2_helper as helper2
import pandas as pd

import collections 

# filters permutations that consist of only gaps ('-')
def filter_all_gaps(perms):
    filtered_perms = []
    for perm in perms:
        if perm.count('-') != len(perm):
            filtered_perms.append(perm)
    return filtered_perms

# scores alignments with 1 point per match of each pair of sequences
# for example, if there is an alignment for 3 sequences, 2 points is the score
def score_alignment(alignment):
    counter = dict(collections.Counter(alignment))
    score = 0
    for element, freq in counter.items():
        if element != '-' and freq > 1:
            score += freq - 1
    return score

# get all possible permutations for gaps and nucleotides
def get_permutations(chars, n):
    permutations = [[c] + ['-']*(len(chars)-1) for c in chars]
    if n == 2:
        permutations = [[i, j] for i in permutations[0] for j in permutations[1]]
    elif n == 3: 
        permutations = [[i, j, k] for i in permutations[0] for j in permutations[1] for k in permutations[2]]
    elif n == 4:
        permutations = [[i, j, k, v] for i in permutations[0] for j in permutations[1] for k in permutations[2] for v in permutations[3]]
    elif n == 5: 
        permutations = [[i, j, k, v, w] for i in permutations[0] for j in permutations[1] for k in permutations[2] for v in permutations[3] for w in permutations[4]]
    else:
        print("number of sequences not supported! Likely to crash!")
    permutations = filter_all_gaps(permutations)
    return permutations

### Algorithm

In [91]:
import numpy as np

def align_dynamic_generic2(seqs, verbose=False):
    
    if verbose:
        print(f"Aligning the following sequences: {seqs}")
    
    # final alignment idx (where solution will be)
    result_idx = tuple([len(s) for s in seqs])
    
    # initializing matrices
    shape = tuple([len(s) + 1 for s in seqs])
    scores = np.zeros(shape=shape) # score matrix
    aligned = {}  # alignment matrix
    
    it = np.nditer(scores, flags=['multi_index']) 
    
    # iterarating over multi-dimensional matrix
    for score in it:
        idxs = it.multi_index
        
        # 'n' is boilerplate for nucleotide, indicating not a gap
        chars = ['n']*len(seqs)

        # finding all nucleotide and gap permutations
        permutations = get_permutations(chars, n=len(seqs))
        aligned_scores = []
        aligned_possibilities = []

        # scoring and aligning all different permutations
        for p in permutations:
            
            # figure out indexes to look back in table
            look_idxs = []
            for numb in range(len(p)):
                
                # get nucleotide from i-1 (insert nucleotide)
                if p[numb] == 'n':
                    look_idxs.append(idxs[numb] - 1 if idxs[numb] > 1 else 0)
                    
                # get nucleotide from i (insert gap)
                else:
                    look_idxs.append(idxs[numb])
            
            # convert to tuple for indexing 
            look_idxs = tuple(look_idxs)
            
            # getting initial look back starting alignment
            try:
                opt = list(aligned[look_idxs])
            except KeyError:
                aligned[look_idxs] = ["" for i in range(len(seqs))]
                opt = list(aligned[look_idxs])
            
            alignment = []
            
            # appending new characters to alignment
            for numb in range(len(look_idxs)):
                # gap
                if look_idxs[numb] == idxs[numb]:
                    opt[numb] += '-'
                    alignment.append('-')
                
                # nucleotide
                else: 
                    opt[numb] += seqs[numb][look_idxs[numb]]
                    alignment.append(seqs[numb][look_idxs[numb]])
            
            # getting initial score
            score = scores[look_idxs]
    
            # scoring new alignment characters
            score += score_alignment(alignment)

            aligned_possibilities.append(opt)
            aligned_scores.append(score)
            
            if verbose:
                print(f"idxs: {idxs}\np: {p}\nlook_idxs: {look_idxs}\n\n")

        max_score = max(aligned_scores)
        scores[idxs] = max_score
        
        # tie-breaking of same scores (priorizes nucleotides over gaps)
        max_score_idxs = {}
        for numb in range(len(aligned_scores)):
            if aligned_scores[numb] == max_score:
                c_count = 0
                for aligned_seq in aligned_possibilities[numb]:
                    for c in aligned_seq:
                        if c != '-':
                            c_count += 1
                max_score_idxs[numb] = c_count
        chosen_idx = max(max_score_idxs, key=max_score_idxs.get)
        
        chosen_aligned = aligned_possibilities[chosen_idx]
        aligned[idxs] = chosen_aligned
        
        if verbose:
            print(aligned_possibilities, aligned_scores)
            print(f"chosen aligned: {aligned[idxs]}, chosen score: {scores[idxs]}")
            print(f"aligned matrix: {aligned}")
            print(f"score matrix: {scores}")
            print()
    
    # removing trailing dashes from final alignment
    final_alignment = [s[1:] for s in aligned[result_idx]]
    final_score = scores[result_idx]
    
    return final_score, final_alignment

In [92]:
align_dynamic_generic2(['AT', 'AGT'], verbose=False)

(2.0, ['A-T', 'AGT'])

In [93]:
align_dynamic_generic2(['AT', 'AGT', 'AGTCG'], verbose=False)

(5.0, ['A-T--', 'AGT--', 'AGTCG'])

In [94]:
align_dynamic_generic2(['A', 'A', 'GT'], verbose=False)

(1.0, ['-A', '-A', 'GT'])

In [95]:
align_dynamic_generic2(['AT', 'AGT', 'AGTCG', 'GT'], verbose=False)

(7.0, ['A-T--', 'AGT--', 'AGTCG', '-GT--'])

In [96]:
align_dynamic_generic2(['AT', 'AGT', 'AGTCG', 'CG', 'GT'], verbose=False)

(9.0, ['A-T--', 'AGT--', 'AGTCG', '---CG', '-GT--'])

### Old Approaches (Archive Reasons)

In [8]:
# 3 Sequences Alignment

<img src="https://bioinformaticsalgorithms.com/images/Alignment/multiple_alignment_recurrence.png" width=500>

In [9]:
# hardcoded 3D alignment
def align_dynamic(seqs, verbose=False):
    s1 = seqs[0]
    s2 = seqs[1]
    s3 = seqs[2]
    scores = []
    aligned = []
    for i in range(len(s1)+1):
        i_score = []
        i_aligned = []
        for j in range(len(s2)+1):
            j_score = []
            j_aligned = []
            for k in range(len(s3)+1):
                j_score.append(0)
                j_aligned.append(("", "", ""))
            i_score.append(j_score)
            i_aligned.append(j_aligned)
        scores.append(i_score)
        aligned.append(i_aligned)
        
    for i in range(1,len(s1)+1):
        for j in range(1,len(s2)+1):
            for k in range(1,len(s3)+1):
                # 1: si-1, j, k + score(vi, -, -)
                opt1_s1 = aligned[i-1][j][k][0] 
                opt1_s2 = aligned[i-1][j][k][1] 
                opt1_s3 = aligned[i-1][j][k][2]
                score1 = scores[i-1][j][k]
                opt1_aligned = (opt1_s1 + s1[i-1], opt1_s2 + '-',  opt1_s3 + '-')
                
                # 2: si, j-1, k + score(-, wj, -)
                opt2_s1 = aligned[i][j-1][k][0] 
                opt2_s2 = aligned[i][j-1][k][1] 
                opt2_s3 = aligned[i][j-1][k][2]
                score2 = scores[i][j-1][k]
                opt2_aligned = (opt2_s1 + '-', opt2_s2 + s2[j-1],  opt2_s3 + '-')
                
                # 3: si, j, k-1 + score(-, -, uk)
                opt3_s1 = aligned[i][j][k-1][0] 
                opt3_s2 = aligned[i][j][k-1][1] 
                opt3_s3 = aligned[i][j][k-1][2]
                score3 = scores[i][j][k-1]
                opt3_aligned = (opt3_s1 + '-', opt3_s2 + '-',  opt3_s3 + s3[k-1])
                
                # 4:  si-1, j-1, k + score(vi, wj, -)
                opt4_s1 = aligned[i-1][j-1][k][0] 
                opt4_s2 = aligned[i-1][j-1][k][1] 
                opt4_s3 = aligned[i-1][j-1][k][2]
                score4 = scores[i-1][j-1][k]
                if s1[i-1] == s2[j-1]:
                    score4 += 1 
                opt4_aligned = (opt4_s1 + s1[i-1], opt4_s2 + s2[j-1],  opt4_s3 + '-')
                
                # 5: si-1, j, k-1 + score(vi, -, uk)
                opt5_s1 = aligned[i-1][j][k-1][0] 
                opt5_s2 = aligned[i-1][j][k-1][1] 
                opt5_s3 = aligned[i-1][j][k-1][2]
                score5 = scores[i-1][j][k-1]
                if s1[i-1] == s3[k-1]:
                    score5 += 1
                opt5_aligned = (opt5_s1 + s1[i-1], opt5_s2 + '-',  opt5_s3 + s3[k-1])
                
                # 6: si, j-1, k-1 + score(-, wj, uk)
                opt6_s1 = aligned[i][j-1][k-1][0] 
                opt6_s2 = aligned[i][j-1][k-1][1] 
                opt6_s3 = aligned[i][j-1][k-1][2]
                score6 = scores[i][j-1][k-1]
                if s2[j-1] == s3[k-1]:
                    score6 += 1
                opt6_aligned = (opt6_s1 + '-', opt6_s2 + s2[j-1],  opt6_s3 + s3[k-1])
                
                # 7: si-1, j-1, k-1 + score(vi, wj, uk)
                opt7_s1 = aligned[i-1][j-1][k-1][0] 
                opt7_s2 = aligned[i-1][j-1][k-1][1] 
                opt7_s3 = aligned[i-1][j-1][k-1][2]
                score7 = scores[i-1][j-1][k-1]
                if s1[i-1] == s2[j-1] and s2[j-1] == s3[k-1]:
                    score7 += 2
                elif s1[i-1] == s2[j-1] or s2[j-1] == s3[k-1] or s3[k-1]:
                    score7 += 1
                    
                opt7_aligned = (opt7_s1 + s1[i-1], opt7_s2 + s2[j-1],  opt7_s3 + s3[k-1])
                
                all_scores = [score1, score2, score3, score4, score5, score6, score7]
                max_score = max(all_scores)
                scores[i][j][k] = max_score
                
                if max_score == score1 and opt1_aligned != ('-', '-', '-'):
                    aligned[i][j][k] = opt1_aligned
                elif max_score == score2 and opt2_aligned != ('-', '-', '-'):
                    aligned[i][j][k] = opt2_aligned
                elif max_score == score3 and opt3_aligned != ('-', '-', '-'):
                    aligned[i][j][k] = opt3_aligned
                elif max_score == score4 and opt4_aligned != ('-', '-', '-'):
                    aligned[i][j][k] = opt4_aligned
                elif max_score == score5 and opt5_aligned != ('-', '-', '-'):
                    aligned[i][j][k] = opt5_aligned
                elif max_score == score6 and opt6_aligned != ('-', '-', '-'):
                    aligned[i][j][k] = opt6_aligned
                else:
                    aligned[i][j][k] = opt7_aligned
                if verbose:
                    print(f"opt1_aligned: {opt1_aligned}")
                    print(f"opt2_aligned: {opt2_aligned}")
                    print(f"opt3_aligned: {opt3_aligned}")
                    print(f"opt4_aligned: {opt4_aligned}")
                    print(f"opt5_aligned: {opt5_aligned}")
                    print(f"opt6_aligned: {opt6_aligned}")
                    print(f"opt7_aligned: {opt7_aligned}")
                    print(f"chosen aligned: {aligned[i][j][k]}, chosen score: {scores[i][j][k]}")

    return scores[i][j][k], aligned[i][j][k]

In [10]:
# align_dynamic(['ATGTTATA', 'AGCGATCA', 'ATCGTCTC'])
align_dynamic(['AT', 'AGT', 'AGTCG'], verbose = False)

(5, ('A-T--', 'AGT--', 'AGTCG'))

In [11]:
# use nd_arrays
def align_dynamic_generic(seqs, verbose=False):
    s1 = seqs[0]
    s2 = seqs[1]
    s3 = seqs[2]
    scores = []
    aligned = []
    for i in range(len(s1)+1):
        i_score = []
        i_aligned = []
        for j in range(len(s2)+1):
            j_score = []
            j_aligned = []
            for k in range(len(s3)+1):
                j_score.append(0)
                i_part = ""
                j_part = ""
                k_part = ""
                if i == 0:
                    i_part = '-'
                if j == 0:
                    j_part = '-'
                if k == 0:
                    k_part = '-'
                    
                j_aligned.append((i_part, j_part, k_part))
            i_score.append(j_score)
            i_aligned.append(j_aligned)
        scores.append(i_score)
        aligned.append(i_aligned)
        
    for i in range(1,len(s1)+1):
        for j in range(1,len(s2)+1):
            for k in range(1,len(s3)+1):
                # 'n' is boilerplate for nucleotide, indicating not a gap
                chars = ['n', 'n', 'n']
                
                # finding all nucleotide and gap permutations
                permutations = [[c] + ['-']*(len(chars)-1) for c in chars]
                permutations = [[i, j, k] for i in permutations[0] for j in permutations[1] for k in permutations[2]]
                permutations = filter_all_gaps(permutations)
                aligned_scores = []
                aligned_possibilities = []
                    
                for p in permutations:
                    p_i = p[0]
                    p_j = p[1]
                    p_k = p[2]
                    opt = None
                    score = None
                    # i-row
                    if p_i == 'n':
                        opt = aligned[i-1]
                        score = scores[i-1]
                    else: 
                        opt = aligned[i]
                        score = scores[i]
                      
                    # j-row
                    if p_j == 'n':
                        opt = opt[j-1]
                        score = score[j-1]
                    else: 
                        opt = opt[j]
                        score = score[j]
                        
                    # k-row
                    if p_k == 'n':
                        opt = opt[k-1]
                        score = score[k-1]
                    else: 
                        opt = opt[k]
                        score = score[k]
                        
                    if opt == ('-', '-', '-'):
                        opt = ('', '', '')
                        
                    opt_i = opt[0] + s1[i-1] if p_i != '-' else opt[0] + p_i
                    opt_j = opt[1] + s2[j-1] if p_j != '-' else opt[1] + p_j
                    opt_k = opt[2] + s3[k-1] if p_k != '-' else opt[2] + p_k
                    
                    alignment = [opt_i[-1], opt_j[-1], opt_k[-1]]
                    score += score_alignment(alignment)
                    
                    aligned_possibilities.append([opt_i, opt_j, opt_k])
                    aligned_scores.append(score)
                
                max_score = max(aligned_scores)
                scores[i][j][k] = max_score
                chosen_aligned = aligned_possibilities[aligned_scores.index(max_score)]
                aligned[i][j][k] = chosen_aligned
                if verbose:
                    print(aligned_possibilities, aligned_scores)
                    print(f"chosen aligned: {aligned[i][j][k]}, chosen score: {scores[i][j][k]}")
                    print()
                    
    return scores[i][j][k], aligned[i][j][k]

In [12]:
align_dynamic_generic(['AT', 'AGT', 'AGTCG'], verbose=False)

(5, ['A-T--', 'AGT--', 'AGTCG'])